In [1]:
import json
import os
import time
import pickle
import random

from instagramy import InstagramUser
from instagramy import InstagramHashTag
from instagramy import InstagramPost

from nedima.utils import adls_utils
from nedima.utils import groundswell as gs

In [2]:
secrets = adls_utils.load_secrets()
adls_client = adls_utils.initialize_storage_account_ad()
fs_client = adls_client.get_file_system_client(file_system=secrets['azure']['adls_fs_landing'])

In [3]:
#secrets

In [4]:
#tag_dated, sleep_time = adls_utils.load_inspection_snapshot()
#for idx, val in enumerate(gs.trim_top_posts(tag_dated.top_posts, delta_seconds = 45)):
#    print(idx, val.upload_time, val.shortcode)

In [1]:
while(True):
    try:
        tag_dated, sleep_time = adls_utils.load_inspection_snapshot()
    except:
        tag_dated = InstagramHashTag('surf', sessionid=random.choice(secrets['instagram']['session_id']))
        sleep_time = 30 + gs.calculate_waiting_time(tag_dated, tag_dated)
        print("[SNAPSHOT] The snapshot couldn't be loaded. A new snapshot will be generated. Some posts may have gotten lost")
        print("[SNAPSHOT] {} seconds remaining until next inspection".format(sleep_time))
    
    time.sleep(sleep_time)

    random_id = random.choice(secrets['instagram']['session_id'])
    print("[SESSION] Randomly selected the sessionid {} for the next inspection".format(random_id))
    tag_latest = InstagramHashTag('surf', sessionid=random_id)

    dump_dict = {
        "posts_short" : gs.get_diff_top_posts(tag_latest, tag_dated),
        "posts_full" : gs.get_diff_json(tag_latest, tag_dated, False)
    }

    adls_utils.dump_adls_json(dump_dict, tag_latest, fs_client)

    adls_utils.dump_inspection_snapshot(tag_latest, tag_dated)
    print("")

NameError: name 'InstagramHashTag' is not defined